<a href="https://colab.research.google.com/github/ngohonggiang/Echocardiography/blob/main/Ph%C3%A2n_lo%E1%BA%A1i_%E1%BA%A3nh_si%C3%AAu_%C3%A2m_tim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import cv2
import numpy as np
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from collections import namedtuple
from sklearn.metrics import classification_report

In [2]:
!git clone https://github.com/ngohonggiang/Echocardiography.git

Cloning into 'Echocardiography'...
remote: Enumerating objects: 8332, done.
remote: Counting objects: 100% (8332/8332), done.
remote: Compressing objects: 100% (8329/8329), done.
remote: Total 8332 (delta 3), reused 8328 (delta 2), pack-reused 0
Receiving objects: 100% (8332/8332), 488.02 MiB | 26.85 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (8327/8327), done.


In [11]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])

def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

def prepare_data():
  size = 224
  transform_train = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.ToTensor()
  ])
  transform_test = transforms.Compose([
    transforms.Resize((size, size)),                                 
    transforms.ToTensor()
  ])
  trainset = torchvision.datasets.ImageFolder(
      root='/content/Echocardiography/DATA_CHAMBER_2021/train', transform=transform_train)
  testset = torchvision.datasets.ImageFolder(
      root='/content/Echocardiography/DATA_CHAMBER_2021/test', transform=transform_test)
  return TrainTest(train=trainset, test=testset)

def prepare_loader(datasets):
  batch_size = 32
  trainloader = DataLoader(dataset=datasets.train, batch_size=batch_size, shuffle=True, num_workers=2)
  testloader = DataLoader(dataset=datasets.test, batch_size=batch_size, shuffle=False, num_workers=2)
  return TrainTest(train=trainloader, test=testloader)


In [12]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_step = 40
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    
    outputs = model(images)
    loss = loss_func(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    running_loss += loss.item()
    if i % reporting_step == reporting_step - 1:
        print(f"Epoch {epoch} Step {i} ave_loss {running_loss/reporting_step:0.4f}")
        running_loss = 0.0

def test_epoch(epoch, model, loader, device):
  pred = []
  true = []
  with torch.no_grad():
    model.eval()
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)

      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      pred += list(predicted.cpu().numpy())
      true += list(labels.cpu().numpy())
  return pred, true

In [13]:
def main(model=None):
  classes = get_classes()
  datasets = prepare_data()
  loaders = prepare_loader(datasets)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print(device)

  if model == 'vgg16':
    PATH = './vgg16.pth'
    model = torchvision.models.vgg16()
    model.classifier[6] = torch.nn.modules.Linear(in_features=4096, out_features=3)
  elif model == 'vgg19':
    PATH = './vgg19.pth'
    model = torchvision.models.vgg19()
    model.classifier[6] = torch.nn.modules.Linear(in_features=4096, out_features=3)
  elif model == 'resnet50':
    PATH = './resnet50.pth'
    model = torchvision.models.resnet50()
    model.fc = torch.nn.modules.Linear(in_features=2048, out_features=3)

  model.to(device)
  loss_function = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

  for epoch in range(8):
    train_epoch(epoch, model, loaders.train, loss_function, optimizer, device)
    pred, true = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(true, pred, target_names=classes))  
    print("---------------------------------------------------------------")      

    torch.save(model.state_dict(), PATH)
  return model

In [14]:
modelVGG16 = main(model='vgg16')

cuda:0
Epoch 0 Step 39 ave_loss 1.0948
Epoch 0 Step 79 ave_loss 1.0031
Epoch 0 Step 119 ave_loss 0.9704
Epoch 0 Step 159 ave_loss 0.6134
Epoch 0 Step 199 ave_loss 0.3671
              precision    recall  f1-score   support

          2C       0.49      0.86      0.62       409
          3C       0.73      0.90      0.81       367
          4C       1.00      0.52      0.69       831

    accuracy                           0.69      1607
   macro avg       0.74      0.76      0.71      1607
weighted avg       0.81      0.69      0.70      1607

---------------------------------------------------------------
Epoch 1 Step 39 ave_loss 0.3008
Epoch 1 Step 79 ave_loss 0.1591
Epoch 1 Step 119 ave_loss 0.1438
Epoch 1 Step 159 ave_loss 0.0927
Epoch 1 Step 199 ave_loss 0.1411
              precision    recall  f1-score   support

          2C       0.91      0.97      0.94       409
          3C       0.89      0.94      0.92       367
          4C       1.00      0.94      0.97       831

    